# Buckling under radial force: realistic (beam) spokes

In [ ]:
import bikewheellib as bl
from doetools import RadialBucklingDOE, wheel_library
from shutil import copy2
from imp import find_module

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib inline

In [ ]:
# Set default colors
default_colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
                  '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
                  '#bcbd22', '#17becf']

mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=default_colors)

## ABAQUS numerical experiment

In [ ]:
beam_dir = '../data/doe/doe_rad_buckling_beam'

if True:
    doe = RadialBucklingDOE(out_dir=beam_dir,
                            opts={'spk_paired': False,
                                  'spk_eltype': 'beam',
                                  'sim_type': 'exp',
                                  'sim_u2': 0.01})

    wheel_types = ['cheap_MTB', 'vintage_rd', 'racing_700',
                   'folding', 'track', 'tandem', 'high_wheel']

    for w_type in wheel_types:
        
        wheel = wheel_library.create_predefined_wheel(w_type)
        
        Tc, nc = bl.calc_buckling_tension(wheel)

        for Tn in [0.0, 0.10, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]:
            
            wheel.apply_tension(Tn*Tc)
            
            jobname = '{:s}_{:.2f}'.format(w_type, Tn)
            doe.add_experiment(wheel, opts={'jobname': jobname,
                                            'wheel_type': w_type,
                                            'spk_Tn': Tn,
                                            'spk_T': Tn * Tc})

    print('\nCreated {0:d} simulations'.format(len(doe.db)))

    doe.write_input_files(N_batches=4)
    doe.to_csv()
    
    # Copy postprocessing script
    copy2(src=find_module('doetools')[1] + '/postproc_rad_buckling.py',
          dst=beam_dir)

In [ ]:
# Load DOE database
print 'Loading database...'
doe = RadialBucklingDOE(out_dir=beam_dir, db_file=beam_dir+'/_doe_db.csv')

# Populate DOE database with results
print 'Extracting results...'
doe.extract_results()
print '\nDone'

In [ ]:
# Calculate normalized buckling load
f = lambda r: r['Pc_max'] / doe.db[(doe.db['wheel_type'] == r['wheel_type']) &
                                   (doe.db['spk_Tn'] == 0.0)]['Pc_max'][0]

doe.db['Pc_max_norm'] = doe.db.apply(f, axis=1)

In [ ]:
cp = sns.color_palette('Reds', 4)

with plt.style.context('seaborn-paper'):
    
    fig, ax = plt.subplots(ncols=2, figsize=(5., 2.5))
    
    d = doe.db[(doe.db['wheel_type'] == 'vintage_rd') & doe.db['spk_Tn'].isin([0.2, 0.4, 0.6, 0.8])]
    
    c = 0
    for i, r in d.iterrows():
        pd_data = pd.read_csv(beam_dir + '/' + r.name + '_collapse_Pd.csv')

        i_Pc = np.argmin(np.abs(pd_data['RF2 [N]'] - r['Pc_max']))

        ax[0].plot(1000*pd_data['U2 [m]'], pd_data['RF2 [N]']/1000,
                   label=r'$T/T_c$ = {:.1f}'.format(r['spk_Tn']), color=cp[c])
        c = c + 1
        ax[0].plot(1000*pd_data['U2 [m]'][i_Pc], pd_data['RF2 [N]'][i_Pc]/1000, 'ko', label='_no_legend_')

    ax[0].set_xlim([0., 8.])
    ax[0].set_xticks(range(0, 9, 2))
    
    ax[0].set_ylim([0., 5.])
    ax[0].set_yticks(range(6))

    ax[0].set_xlabel('Radial displacement [mm]')
    ax[0].set_ylabel('Radial load [kN]')
    
    
    mrk = ['o', '*', 'd', 's', '^', 'p', 'v']
    type_order = doe.db[['wheel_type', 'Pc_nb']].groupby('wheel_type').max().sort_values(by='Pc_nb').index

    for i, w in enumerate(type_order):
        d = doe.db[doe.db['wheel_type'] == w]

        plt.plot(d['spk_Tn'], d['Pc_max_norm'], label=w.replace('_', ' '),
                 marker=mrk[i], markersize=6)

    ax[1].set_ylim([0., 2.]);
    ax[1].set_xlim([0., 1.]);
    ax[1].set_xticks([0., 0.25, 0.5, 0.75, 1])
    ax[1].set_xticklabels(['0', '0.25', '0.5', '0.75', '1'])

    ax[1].set_xlabel(r'\$T/T_c\$')
    ax[1].set_ylabel(r'\$P_c / P_{c,0}\$')
    
    ax[1].legend()
    
    plt.tight_layout()
    plt.savefig('../figs/buckling_ext_loads/_python_buckling_rad_beam.pdf')